# Load data

In [15]:
import pandas as pd
import os
from pathlib import Path
from pprint import pprint
import sys
sys.path.append("..")


#get the path to the data
data_path = Path("..") / "data"

#get the list of parquet files
parquet_files = list(data_path.glob("*.parquet"))
print(parquet_files)
#load example from train parquet
df = pd.read_parquet(parquet_files[0])
df.head()

[PosixPath('../data/train.parquet'), PosixPath('../data/test.parquet'), PosixPath('../data/val.parquet')]


,Context,Question,Program,Answer
0,# UNITED\tSTATES\n\n# SECURITIES\tAND\tEXCHANG...,what is the the interest expense in 2009?,interest_expense = 100 / 100\nannual_interest_...,380
1,# UNITED\tSTATES\tSECURITIES\tAND\tEXCHANGE\tC...,what was the total operating expenses in 2018 ...,,41932
2,# UNITED\tSTATES\tSECURITIES\tAND\tEXCHANGE\tC...,what percentage of total cash and investments ...,available_for_sale_investments = 14001\ntotal_...,53%
3,# UNITED\tSTATES\n\n# SECURITIES\tAND\tEXCHANG...,what is the growth rate in net revenue in 2008?,net_revenue_2008 = 959.2\nnet_revenue_2007 = 9...,-3.2%
4,10-K\t1\tcitigroup_10k.htm\tANNUAL\tREPORT\n\n...,what was the growth rate of the loans held-for...,,56.25%


# Preprocess data

In [16]:
sample = df['Context'][0]

from src.text_preprocessing import TenKPreprocessor

preprocessor = TenKPreprocessor()

pprint(preprocessor.preprocess(sample))

{'doc_id': None,
 'sections': {'ITEM 12': '| Period | Total Number ofSharesPurchased(a) | '
                         'Average Price PaidPer Share(b) | Total Number of '
                         'SharesPurchased as Part ofPublicly AnnouncedPlans or '
                         'Programs(c) | Value of Shares thatMay Yet Be '
                         'PurchasedUnder the Plans orPrograms |\n'
                         '| :--- | :--- | :--- | :--- | :--- |\n'
                         '| August\\t2,\\t2009\\tthrough\\tAugust\\t29,\\t2009 '
                         '| — | — | — | $91,614,067 |\n'
                         '| '
                         'August\\t30,\\t2009\\tthrough\\tSeptember\\t26,\\t2009 '
                         '| 1,017 | $28.39 | — | $91,614,067 |\n'
                         '| '
                         'September\\t27,\\t2009\\tthrough\\tOctober\\t31,\\t2009 '
                         '| 667 | $28.16 | — | $91,614,067 |\n'
                         '| Total | 1,684 | $28.3

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_id = "nlpaueb/sec-bert-shape"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(model_id)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


/Users/jon/GitHub/dowjones-takehome/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForTokenClassification were not initialized from the model checkpoint at nlpaueb/sec-bert-shape and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
